In [1]:
import jax
from jax.config import config
config.update("jax_enable_x64", True)

In [2]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import moscot
from moscot.problems.time import TemporalProblem
import moscot.plotting as mpl
import pandas as pd
import os

sc.set_figure_params(scanpy=True, dpi=80, dpi_save=200)
                         
import mplscience

mplscience.available_styles()
mplscience.set_style(reset_current=True)
plt.rcParams['legend.scatterpoints'] = 1 

['default', 'despine']


In [3]:
output_dir = "."

In [4]:
adata = sc.read_h5ad("/lustre/groups/ml01/workspace/moscot_paper/pancreas/adata_processed.h5ad")

In [5]:
chromvar_features = pd.read_csv("/lustre/groups/ml01/workspace/moscot_paper/pancreas/cisBP_chromvar_annotations_reduced.csv")

In [6]:
chromvar_features.set_index("Unnamed: 0", inplace=True)

In [7]:
chromvar_features = chromvar_features.T

In [8]:
chromvar_features.head()

Unnamed: 0,new motif,new motif.1,new motif.2,new motif.3,new motif.4,new motif.5,new motif.6,new motif.7,new motif.8,new motif.9,...,new motif.6004,new motif.6005,new motif.6006,new motif.6007,new motif.6008,new motif.6009,new motif.6010,new motif.6011,new motif.6012,new motif.6013
E14-5_AAACAGCCAACAGCCT-1,1.449150,-1.548024,0.127186,-0.401432,5.779336,-0.047665,-0.104796,0.082138,-0.065305,-0.163256,...,-1.134490,0.531467,-0.404171,-2.334780,0.179397,-0.388121,0.772678,0.357471,0.428473,0.772678
E14-5_AAACAGCCAACCCTCC-1,-0.986286,3.096357,2.472101,0.594340,5.514991,1.100158,-0.874710,-0.670196,0.534449,-0.349137,...,-1.157848,-1.993996,0.308834,-1.702847,-1.646581,1.172381,0.286935,-0.433321,2.500077,0.286935
E14-5_AAACAGCCACCTGTAA-1,2.217122,-0.694262,-1.776413,-0.556984,-0.999821,-1.439918,-0.359144,0.231731,-1.116241,-0.331855,...,1.615487,-0.255339,-1.269120,2.926865,0.709345,0.389420,-1.620937,-1.635631,-0.787535,-1.620937
E14-5_AAACAGCCACTAAGCC-1,1.043075,1.337077,-0.491503,1.104463,5.234869,-0.295337,0.489305,-1.783421,-1.110079,-0.149686,...,-0.439479,-0.452744,-1.027292,-2.424418,-0.645681,-0.100782,2.136637,0.185030,0.231816,2.136637
E14-5_AAACAGCCAGGATAAC-1,0.537357,-0.343018,-1.047688,-1.379141,0.126564,-0.931787,-0.365652,-1.572852,-1.259440,-1.114664,...,0.704474,-0.813751,-1.530833,-0.959602,-1.077125,-1.498234,1.009304,1.857225,1.016590,1.009304


In [9]:
chromvar_features["old_index"] = chromvar_features.index

In [10]:
def rename_index(x):
    split = x["old_index"].split("_")
    if split[0] == "E14-5":
        return split[1]+"-0"
    if split[0] == "E15-5":
        return split[1]+"-1"
    raise ValueError("Pattern does not match")

In [11]:
chromvar_features["index_adapted"] = chromvar_features.apply(rename_index ,axis=1)

In [16]:
chromvar_features.head()

Unnamed: 0,new motif,new motif.1,new motif.2,new motif.3,new motif.4,new motif.5,new motif.6,new motif.7,new motif.8,new motif.9,...,new motif.6006,new motif.6007,new motif.6008,new motif.6009,new motif.6010,new motif.6011,new motif.6012,new motif.6013,old_index,index_adapted
E14-5_AAACAGCCAACAGCCT-1,1.449150,-1.548024,0.127186,-0.401432,5.779336,-0.047665,-0.104796,0.082138,-0.065305,-0.163256,...,-0.404171,-2.334780,0.179397,-0.388121,0.772678,0.357471,0.428473,0.772678,E14-5_AAACAGCCAACAGCCT-1,AAACAGCCAACAGCCT-1-0
E14-5_AAACAGCCAACCCTCC-1,-0.986286,3.096357,2.472101,0.594340,5.514991,1.100158,-0.874710,-0.670196,0.534449,-0.349137,...,0.308834,-1.702847,-1.646581,1.172381,0.286935,-0.433321,2.500077,0.286935,E14-5_AAACAGCCAACCCTCC-1,AAACAGCCAACCCTCC-1-0
E14-5_AAACAGCCACCTGTAA-1,2.217122,-0.694262,-1.776413,-0.556984,-0.999821,-1.439918,-0.359144,0.231731,-1.116241,-0.331855,...,-1.269120,2.926865,0.709345,0.389420,-1.620937,-1.635631,-0.787535,-1.620937,E14-5_AAACAGCCACCTGTAA-1,AAACAGCCACCTGTAA-1-0
E14-5_AAACAGCCACTAAGCC-1,1.043075,1.337077,-0.491503,1.104463,5.234869,-0.295337,0.489305,-1.783421,-1.110079,-0.149686,...,-1.027292,-2.424418,-0.645681,-0.100782,2.136637,0.185030,0.231816,2.136637,E14-5_AAACAGCCACTAAGCC-1,AAACAGCCACTAAGCC-1-0
E14-5_AAACAGCCAGGATAAC-1,0.537357,-0.343018,-1.047688,-1.379141,0.126564,-0.931787,-0.365652,-1.572852,-1.259440,-1.114664,...,-1.530833,-0.959602,-1.077125,-1.498234,1.009304,1.857225,1.016590,1.009304,E14-5_AAACAGCCAGGATAAC-1,AAACAGCCAGGATAAC-1-0


In [19]:
adata

AnnData object with n_obs × n_vars = 9365 × 242922

In [12]:
endocrine_celltypes = [
    "Ngn3 low",
    "Ngn3 high",
    "Ngn3 high cycling",
    "Fev+",
    "Fev+ Alpha",
    "Fev+ Beta",
    "Fev+ Delta",
    "Eps. progenitors",
    "Alpha",
    "Beta",
    "Delta",
    "Epsilon"
]

In [13]:
adata.obs = adata.obs.merge(chromvar_features, left_index=True, right_index=True)

ValueError: Length of passed value for obs_names is 0, but this AnnData has shape: (9365, 242922)

# Heat kernel distances

In [ ]:
import pygsp

In [ ]:
sc.pp.neighbors(adata, use_rep="X_multi_vi", key_added="multi_vi", n_neighbors=30)

In [ ]:
cell_ids_source = adata[adata.obs["time"]==14.5].obs_names
cell_ids_target = adata[adata.obs["time"]==15.5].obs_names

In [ ]:
assert (adata.obs.iloc[:(len(cell_ids_source))].index == cell_ids_source).all()
assert (adata.obs.iloc[(len(cell_ids_source)):].index == cell_ids_target).all()

In [ ]:
G = pygsp.graphs.Graph(adata.obsp["multi_vi_connectivities"])

In [ ]:
G.estimate_lmax()

In [ ]:
filt = pygsp.filters.Heat(G, tau=100)
diffusion_distances = filt.filter(np.eye(len(adata)))
custom_cost = pd.DataFrame(data=-np.log(diffusion_distances[:len(cell_ids_source), len(cell_ids_source):].copy())+1e-15, index=cell_ids_source, columns=cell_ids_target)

In [ ]:
tp0 = TemporalProblem(adata)
tp0 = tp0.prepare("time", joint_attr="X_multi_vi")

In [ ]:
tp0[14.5, 15.5].set_xy(custom_cost, tag="cost_matrix")

In [ ]:
tp0 = tp0.solve(max_iterations=1e8)

In [ ]:
tp0.save("/lustre/groups/ml01/workspace/moscot_paper/pancreas/", file_prefix="cisBP", overwrite=True)